In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [6]:
from sympy import sqrt, cos, sin,  cosh,  sinh, Add
from sympy import Symbol, S, oo, Rational, Function, pi , factorial, O, symbols, lambdify, I
from sympy import Derivative, series, limit, Matrix, simplify, collect, solve, Poly, poly, diff, nsolve, transpose, factor_terms
from sympy import latex, init_printing
from sympy.core.numbers import NegativeOne, One, Zero
from sympy.printing.latex import print_latex

import numpy as np
import pandas as pd
from tqdm import tqdm
import ray
import time

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import defaultdict
from itertools import product, combinations
from functools import reduce
from acchamiltoniansandmatrices.Tracking.LinearMatrixTracking import nestList, GenerateNDimCoordinateGrid
from acchamiltoniansandmatrices.Hamiltonians.LatticeElementHamiltonians import HamDrift6D, HamQuad6D
from acchamiltoniansandmatrices.Hamiltonians.Operators import Poisson
from acchamiltoniansandmatrices.LieMaps.Poisson import PoissonBracket
from acchamiltoniansandmatrices.LieMaps.LieOperator import LieOperator
from acchamiltoniansandmatrices.Utils.Plotting import multi_countour_plot
from acchamiltoniansandmatrices.Factorization.Factorization import truncate, taylorize, dragt_finn_factorization
from acchamiltoniansandmatrices.Utils.Polynomials import getpoly, RotationMatrix2D, explicitCheck, geteigenvects
from acchamiltoniansandmatrices.Matrices.NumpyMatrices import RnpQuad6DChroma
from acchamiltoniansandmatrices.Matrices.SymbMatrices import RsymbQuad6DChroma
from acchamiltoniansandmatrices.Integrators import integrator
from acchamiltoniansandmatrices.Tracking.LinearMatrixTracking import nestList

from acchamiltoniansandmatrices.LieMaps.BCH import generate_bch_expression

In [4]:
x, px, y, py, z, delta = symbols('x p_x y p_y z delta', real=True)
eps, h , k0, k1, k2, sigma, betag = symbols('epsilon h k_0 k_1 k_2 sigma beta_gamma', real=True)
L, beta0, gamma0 = symbols(" L beta_0 gamma_0", real=True, positive=True)
x0,px0 = symbols('x_0 px_0') # to not overwrite anything
p = symbols("p")

In [5]:
H1 = p**2+x**2
H2 = p**2

# Comparing old_BCH with new

In [14]:
X = Function("X")(x,px)
Y = Function("Y")(x,px)

Lo1 = LieOperator(X,[x],[p]) 
Lo2 = LieOperator(Y,[x],[p])

## order 2

In [27]:
Lo1.BCH(Lo2,2).ham

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2

In [26]:
generate_bch_expression(order=2,coords=[x],mom=[px])

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2

## order 3

In [30]:
Lo1.BCH(Lo2,3).ham.expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12

In [31]:
generate_bch_expression(order=3,coords=[x],mom=[px]).expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12

## order 4

In [32]:
Lo1.BCH(Lo2,4).ham.expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 - PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24

In [33]:
generate_bch_expression(order=4,coords=[x],mom=[px]).expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 - PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24

## order 5

In [37]:
Lo1.BCH(Lo2,5).ham.expand(commutator=True)

X(x, p_x) + Y(x, p_x) - PoissonBracket(X(x, p_x)/720, PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x))) - PoissonBracket(X(x, p_x)/360, PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)), Y(x, p_x))) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 + PoissonBracket(Y(x, p_x)/720, PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)), Y(x, p_x))) - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 - PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24 - PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x)), Y(x, p_x))/360 + PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x)), X(x, p_x))/120 - PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x))

In [38]:
generate_bch_expression(order=5,coords=[x],mom=[px]).expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 - PoissonBracket(-PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), -PoissonBracket(X(x, p_x), Y(x, p_x)))/120 - PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24 - PoissonBracket(-PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)), -PoissonBracket(X(x, p_x), Y(x, p_x)))/360 + PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x)), X(x, p_x))/720 + PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x)), Y(x, p_x))/180 - PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x)), Y(x, p_x))/180 - PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p

# Practical example

In [54]:
Lo1.BCH(Lo2,1).ham.subs(X,H2).subs(Y,H1).doit()

2*p**2 + x**2

In [43]:
generate_bch_expression(order=1,coords=[x],mom=[px]).expand(commutator=True).subs(X,H2).subs(Y,H1)

2*p**2 + x**2

In [55]:
Lo1.BCH(Lo2,2).ham.subs(X,H2).subs(Y,H1).doit()

2*p**2 - 2*p*x + x**2

In [45]:
generate_bch_expression(order=2,coords=[x],mom=[px]).expand(commutator=True).subs(X,H2).subs(Y,H1).doit()

2*p**2 - 2*p*x + x**2

In [56]:
Lo1.BCH(Lo2,3).ham.subs(X,H2).subs(Y,H1).doit()

8*p**2/3 - 2*p*x + x**2

In [46]:
generate_bch_expression(order=3,coords=[x],mom=[px]).expand(commutator=True).subs(X,H2).subs(Y,H1).doit()

2*p**2 - 2*p*x + 5*x**2/3

In [57]:
Lo1.BCH(Lo2,4).ham.subs(X,H2).subs(Y,H1).doit()

8*p**2/3 - 10*p*x/3 + x**2

In [47]:
generate_bch_expression(order=4,coords=[x],mom=[px]).expand(commutator=True).subs(X,H2).subs(Y,H1).doit()

2*p**2 - 10*p*x/3 + 5*x**2/3

In [58]:
Lo1.BCH(Lo2,5).ham.subs(X,H2).subs(Y,H1).doit()

32*p**2/15 - 10*p*x/3 + 23*x**2/15

In [59]:
generate_bch_expression(order=5,coords=[x],mom=[px]).subs(X,H2).subs(Y,H1).doit()

2*p**2 - 2*p*x + x**2

In [63]:
generate_bch_expression(order=6,coords=[x],mom=[px]).subs(X,H2).subs(Y,H1).doit()

2*p**2 - 2*p*x + x**2